In [1]:
## Import necessary packages
import pandas as pd
import numpy as np
import camelot
import warnings
warnings.filterwarnings("ignore")

## Delaware

### Appendix A, Comprehensive PPL

In [32]:
print("Parsing Delaware IUP Appendix A")
# get the full table
de_ppl = camelot.read_pdf("https://www.dhss.delaware.gov/dhss/dph/hsp/files/dwsrfiup2022.pdf", 
                           pages='25-29', flavor='stream', row_tol=1)
print(len(de_ppl))

Parsing Delaware IUP Appendix A
5


In [33]:
# combine all 4 pages into a single file and show length
de_ppl_output = pd.concat([de_ppl[0].df, de_ppl[1].df, de_ppl[2].df,
                           de_ppl[3].df, de_ppl[4].df]).reset_index(drop=True)
print(len(de_ppl_output))
# length is much longer than PPL because rows have been spliced apart, need to correct by recombining them

192


In [112]:
# write out temp file
# de_ppl_output.to_csv("../data/year1/csv/8-Delaware_temp_output.csv", index=False)

# THEN, in excel, manually assign each row a value for a new "key" column 
# such that when it is grouped by, it will combine the rows as they appear in the PDF

# read in manually updated file, which should be the same with one additional column, 'key'
# which represents the rows that should be joined together to rebuild the PDF table
de_ppl_input = pd.read_csv("../data/year1/csv/8-Delaware_temp_input.csv")

In [113]:
# drop empty rows from input file, then replace all NAs with empty string for joining
de_ppl_input = de_ppl_input.fillna('')
de_ppl_input = de_ppl_input.query("key != ''").copy()

In [114]:
de_ppl_input.head(20)

,key,0,1,2,3,4,5,6,7,8,9,10,11,12,13
3,1,,,,,,,,,,,,,,Anticipated to
4,1,,,,,Consolidation to,,,,,,,,,be Bypassed
5,1,,Countryside,,Upgrades and,Artesian - See Adam,,,,,,,State,,for State
6,1,1,Hamlet MHP,66,Interconnect,Gould emails,"$745,470.00",,800,State,,,Grant,"$745,470.00",Funds
7,2,,,,Water System,"New distribution, fire",,,,,,,,,Anticipated to
8,2,,,,Improvements and,"hydrants, water meters",,,,,,,,,be Bypassed
9,2,,Stage Village,,Consolidation with,and connections to,,,,,,,State,,for State
10,2,2,MHP,93,Delmar,Delmar,"$687,800.00",U,730,State,,,Grant,"$687,800.00",Funds
11,3,,,,,"System Upgrades, add",,,,,,,,,
12,3,,Willow Tree,,MHP Water System,storage Kitts,,,,Suppleme,,,Sup,,


In [115]:
# create empty dataframe to assign the group by columns to
de_ppl = pd.DataFrame(columns=['Rank', 'Water System/Borrower', 'Population Served', 'Comprehensive Project Name',
                               'Project Description', 'Amount', 'DAC? - A, E, W, U', 'Total Points',
                               'Funding Appropriation', 'Financing', 'Terms', 'Anticipated Subsidy', 
                               'Anticipated Subsidy Amount', 'Notes'])

# group each column with a function that joins strings with a space across all rows that have the manually created key in common
de_ppl['Rank'] = de_ppl_input.groupby(['key'])['0'].transform(lambda x : " ".join(x).strip())
de_ppl['Water System/Borrower'] = de_ppl_input.groupby(['key'])['1'].transform(lambda x : " ".join(x).strip())
de_ppl['Population Served'] = de_ppl_input.groupby(['key'])['2'].transform(lambda x : " ".join(x).strip())
de_ppl['Comprehensive Project Name'] = de_ppl_input.groupby(['key'])['3'].transform(lambda x : " ".join(x).strip())
de_ppl['Project Description'] = de_ppl_input.groupby(['key'])['4'].transform(lambda x : " ".join(x).strip())
de_ppl['Amount'] = de_ppl_input.groupby(['key'])['5'].transform(lambda x : " ".join(x).strip())
de_ppl['DAC? - A, E, W, U'] = de_ppl_input.groupby(['key'])['6'].transform(lambda x : " ".join(x).strip())
de_ppl['Total Points'] = de_ppl_input.groupby(['key'])['7'].transform(lambda x : " ".join(x).strip())
de_ppl['Funding Appropriation'] = de_ppl_input.groupby(['key'])['8'].transform(lambda x : " ".join(x).strip())
de_ppl['Financing'] = de_ppl_input.groupby(['key'])['9'].transform(lambda x : " ".join(x).strip())
de_ppl['Terms'] = de_ppl_input.groupby(['key'])['10'].transform(lambda x : " ".join(x).strip())
de_ppl['Anticipated Subsidy'] = de_ppl_input.groupby(['key'])['11'].transform(lambda x : " ".join(x).strip())
de_ppl['Anticipated Subsidy Amount'] = de_ppl_input.groupby(['key'])['12'].transform(lambda x : " ".join(x).strip())
de_ppl['Notes'] = de_ppl_input.groupby(['key'])['13'].transform(lambda x : " ".join(x).strip())

In [116]:
de_ppl = de_ppl.drop_duplicates(ignore_index=True)
# drop the consolidated row where all of the former column names ended up after aggregating
de_ppl.reset_index(inplace=True, drop=True)

In [117]:
# because different pages ended up formatting differently based on column values, break up the grouped df into three parts
# to resolve these issues

# the first page is unaffected, splice for re-adding later on
de_ppl_1 = de_ppl.iloc[:14,].reset_index(drop=True).copy()

In [118]:
# starting at row 15-30 (12, Greenwood), Project Description and Amount get combined and need to be split by /n
# then all columns need to be moved over one because of this combination collapsing each value into the wrong column

de_ppl_2 = de_ppl.iloc[15:31,].reset_index(drop=True).copy() 

# moving from right to left, replace the column's values with the column to the left
# until we arrive at the point where data was merged together erroneously
de_ppl_2['Notes'] = de_ppl_2['Anticipated Subsidy Amount']
de_ppl_2['Anticipated Subsidy Amount'] = de_ppl_2['Anticipated Subsidy']
de_ppl_2['Anticipated Subsidy'] = de_ppl_2['Terms']
de_ppl_2['Terms'] = de_ppl_2['Financing']
de_ppl_2['Financing'] = de_ppl_2['Funding Appropriation']
de_ppl_2['Funding Appropriation'] = de_ppl_2['Total Points']
de_ppl_2['Total Points'] = de_ppl_2['DAC? - A, E, W, U']
de_ppl_2['DAC? - A, E, W, U'] = de_ppl_2['Amount']

# split by dollar sign to handle issue where linebreak doesn't always appear
de_ppl_2[['Project Description','Amount']] = de_ppl_2['Project Description'].str.split('$', expand=True)

# add $ back
de_ppl_2['Amount'] = "$" + de_ppl_2['Amount']

# remove linebreak where it appears
de_ppl_2['Project Description'] = de_ppl_2['Project Description'].str.replace("\n", "")

# note that (19, Newark) is the only row where project name and description conflate, need to manually fix
de_ppl_2.iloc[9, 3] = "SWFWTP PFAs Treatment - $4.4M EC"
de_ppl_2.iloc[9, 4] = 'PFA WTP Upgrades'

In [119]:
# keep the second portion of unaffected rows
de_ppl_3 = de_ppl.iloc[31:50,]

In [120]:
# then again at 52 through the end of the df, the DAC column is attached to the Amount column (this only affects 4 values, rest are empty)
# and then all subsequent columns need to be moved over
de_ppl_4 = de_ppl.iloc[51:76,].reset_index(drop=True).copy()

# moving from right to left, replace the column's values with the column to the left
# until we arrive at the point where data was merged together erroneously
de_ppl_4['Notes'] = de_ppl_4['Anticipated Subsidy Amount']
de_ppl_4['Anticipated Subsidy Amount'] = de_ppl_4['Anticipated Subsidy']
de_ppl_4['Anticipated Subsidy'] = de_ppl_4['Terms']
de_ppl_4['Terms'] = de_ppl_4['Financing']
de_ppl_4['Financing'] = de_ppl_4['Funding Appropriation']
de_ppl_4['Funding Appropriation'] = de_ppl_4['Total Points']
de_ppl_4['Total Points'] = de_ppl_4['DAC? - A, E, W, U']

# split Amount into two columns, then fill the NA DAC values
de_ppl_4[['Amount','DAC? - A, E, W, U']] = de_ppl_4['Amount'].str.split('\n', expand=True)
de_ppl_4['DAC? - A, E, W, U'].fillna('', inplace=True)

In [121]:
# bring together individual sections of PPL
de_ppl_final = pd.concat([de_ppl_1, de_ppl_2, de_ppl_3, de_ppl_4])
de_ppl_final.reset_index(drop=True, inplace=True)

# fix some final typos
de_ppl_final['Funding Appropriation'] = de_ppl_final['Funding Appropriation'].str.replace("Suppleme ntal", "Supplemental")
de_ppl_final['Funding Appropriation'] = de_ppl_final['Funding Appropriation'].str.replace("Protectio n", "Protection")

In [122]:
de_ppl_final.to_csv("../data/year1/csv/8-Delaware_PPL.csv", index=False)

## Old File - Kept For Reference

The below code shows the original process for tackling tables where rows are split into multiple rows when going from PDF to df.


In [ ]:
print("Parsing Delaware PPL")
# get the full table
de_ppl = camelot.read_pdf("https://www.dhss.delaware.gov/dhss/dph/hsp/files/dwsrfppl2022.pdf", 
                           pages='1-2', flavor='stream', row_tol=1)
print(len(de_ppl))

In [26]:
## fix lots of split rows manually

# fix Bethany Crest MHP row
de_ppl_output.iloc[5,1] = de_ppl_output.iloc[4,1] + " " + de_ppl_output.iloc[5,1]
de_ppl_output.iloc[5,5] = de_ppl_output.iloc[4,5] + " " + de_ppl_output.iloc[5,5]
de_ppl_output.iloc[5,12] = de_ppl_output.iloc[4,12] + " " + de_ppl_output.iloc[5,12]

# fix Countryside Hamlet MHP row
de_ppl_output.iloc[9,1] = de_ppl_output.iloc[7,1] + " " + de_ppl_output.iloc[8,1]
de_ppl_output.iloc[9,4] = de_ppl_output.iloc[7,4] + " " + de_ppl_output.iloc[8,4]
de_ppl_output.iloc[9,5] = de_ppl_output.iloc[6,5] + " " + de_ppl_output.iloc[7,5] + " " + de_ppl_output.iloc[8,5]
de_ppl_output.iloc[9,9] = de_ppl_output.iloc[8,9]
de_ppl_output.iloc[9,12] = de_ppl_output.iloc[7,12] + " " + de_ppl_output.iloc[8,12]
de_ppl_output.iloc[9,13] = de_ppl_output.iloc[8,13]

# fix 12, Dover row
de_ppl_output.iloc[11,5] = de_ppl_output.iloc[10,5] + " " + de_ppl_output.iloc[11,5]

# fix 22, Dover row
de_ppl_output.iloc[13,5] = de_ppl_output.iloc[12,5] + " " + de_ppl_output.iloc[13,5]

# fix 10, Frederica row
de_ppl_output.iloc[15,4] = de_ppl_output.iloc[14,4] + " " + de_ppl_output.iloc[15,4]

# fix 15, Frederic row
de_ppl_output.iloc[17,12] = de_ppl_output.iloc[16,12] + " " + de_ppl_output.iloc[17,12]

# fix Lewes BPW row
de_ppl_output.iloc[22,5] = de_ppl_output.iloc[21,5] + " " + de_ppl_output.iloc[22,5]

# fix Magnolia row
de_ppl_output.iloc[24,1] = de_ppl_output.iloc[23,1]
de_ppl_output.iloc[24,4] = de_ppl_output.iloc[23,4]
de_ppl_output.iloc[24,5] = de_ppl_output.iloc[23,5]
de_ppl_output.iloc[24,9] = de_ppl_output.iloc[23,9]

# fix 34, Middleton row
de_ppl_output.iloc[26,5] = de_ppl_output.iloc[25,5] + " " + de_ppl_output.iloc[26,5]

# fix 36, Middleton row
de_ppl_output.iloc[27,5] = "Water Main Upgrade"
de_ppl_output.iloc[27,6] = "$1,451,026"

# fix New Castle row
de_ppl_output.iloc[29,5] = "Water Main Replacement"
de_ppl_output.iloc[29,9] = "Supplemental"
de_ppl_output.iloc[29,12] = "Sup DAC"

# fix 19, Newark row
de_ppl_output.iloc[31,4] = de_ppl_output.iloc[30,4] + " " + de_ppl_output.iloc[31,4]
de_ppl_output.iloc[31,12] = de_ppl_output.iloc[30, 12] + " " + de_ppl_output.iloc[31,12]

# fix the second 19, Newwark row 
de_ppl_output.iloc[33,4] = de_ppl_output.iloc[32,4] + " " + de_ppl_output.iloc[33,4]

# fix Sussex Shores row
de_ppl_output.iloc[36,4] = de_ppl_output.iloc[34,4] + " " + de_ppl_output.iloc[35,4] + de_ppl_output.iloc[36,4]
de_ppl_output.iloc[36,5] = de_ppl_output.iloc[34,5] + " " + de_ppl_output.iloc[35,5] + de_ppl_output.iloc[36,5]

# fix 33, Tidewater row
de_ppl_output.iloc[38,4] = de_ppl_output.iloc[37,4] + " " + de_ppl_output.iloc[38,4]
de_ppl_output.iloc[38,5] = de_ppl_output.iloc[37,5] + " " + de_ppl_output.iloc[38,5]

# fix 34, Tidewater row
de_ppl_output.iloc[41,4] = de_ppl_output.iloc[40,4] + " " + de_ppl_output.iloc[41,4]
de_ppl_output.iloc[41,5] = de_ppl_output.iloc[39,5] + " " + de_ppl_output.iloc[40,5] + " " + de_ppl_output.iloc[41,5]


## Manually type out second page because scraping condensing into fewer rows, creating inconsistent assignments

# fix 39, Tidewater row
de_ppl_output.iloc[45,] = ["39", "Tidewater", "DE0000991", "23463", "DelDOT SR 24 Love Creek to Mulberry Knoll", "Join DelDOT and Tidewater SR 24 Reho", "$3,302,000",
"", "180", "Base", "2.00%", "20 years", "", "$0"]

# fix Willow Tree row
de_ppl_output.iloc[49,] = ["3", "Willow Tree MHP", "DE0000134", "141", "MHP Water System Upgrades", "System Upgrades, add storage Kitts Hummock Road", "$1,700,000", "U",
                            "680", "Supplemental Federal Grant", "", "", "Sup DAC", "$1,700,000"]

# fix 6, Wilmington row
de_ppl_output.iloc[54,] = ["6", "Wilmington", "DE0000663", "107976", "LSL Removal", "Water Main and Service line improvements and LSL removal $22.5M LSLR", "$18,501,872", "E, W",
                        "620", "LSLR", "", "", "94% LSLR", "$12,875,792"]

# fix 8, Wilmington row
de_ppl_output.iloc[57,] = ["8", "Wilmington", "DE0000663",	"107976", "Raw Water Transmission Improvements", "From Brandywine Creek to either Porter or Hoopes", "$5,626,800", "E, W",
                        "580", "Supplemental", "", "", "Sup DAC", "$5,626,080"]

# fix 13, Wilmington row
de_ppl_output.iloc[60,] = ["13", "Wilmington", "DE0000663", "107976", "Porter Filter Plant PFAs Removal- $45M project EC", "Construct GAC vessels", "$4,787,405", "E, W",
                          "450", "EC", "", "", "75% EC", "$4,787,405"]

# fix 19, Wilmington row
de_ppl_output.iloc[62,] = ["19", "Wilmington", "DE0000663", "107976", "Clearwell/Storage Improvements at Porter", "Phase I Design application", "$1,200,000", "E, W",
                           "380", "Corpus", "2.00%", "20 years", "", "$0"]


de_ppl_output.iloc[64,] = ["25", "Wilmington", "DE0000663", "107976", "Porter Reliability Improvements", "Porter Reliability Improvements", "$11,000,000", "E, W",
                           "330", "Corpus", "2.00%", "20 years", "", "$0"]

In [27]:
# drop table title rows and reset index
de_ppl_output = de_ppl_output.iloc[5:,].reset_index(drop=True).copy()

In [28]:
# drop rows that aren't projects
de_ppl_output = de_ppl_output.loc[de_ppl_output[0] != ""]

# set columns for all kept rows
de_ppl_output.columns = ["Rank", "Water System/Borrower", "PWSID #", "Population Served", "Fundable Project Name", "Project Description", "Amount",
                         "DAC? - A, E, W, U", "Total Points", "Funding Appropriation", "Financing", "Terms", "Anticipated Subsidy", "Anticipated Subsidy Amount"]


In [29]:
de_ppl_output

,Rank,Water System/Borrower,PWSID #,Population Served,Fundable Project Name,Project Description,Amount,"DAC? - A, E, W, U",Total Points,Funding Appropriation,Financing,Terms,Anticipated Subsidy,Anticipated Subsidy Amount
0,40,Bethany Crest MHP,DE0000628,115,PFAs Treatment,Installation of PFAs resin,"$300,500",,170,State,2.00%,20 years,State Grant,"$300,500"
4,1,Countryside Hamlet MHP,DE0000817,66,Upgrades and Interconnect,Consolidation to Artesian - See Adam Gould emails,"$745,470",,800,State,,,State Grant,"$745,470"
6,12,Dover,DE0000571,38000,LSL inventory,LSL regulatory requirements,"$60,000","E,",480,LSLR,,,,"$60,000"
8,22,Dover,DE0000571,38000,Brandywine Court,LSL inventory and replacement,"$115,000","E,",360,LSLR,2.00%,20 years,,"$115,000"
10,10,Frederica,DE0000587,870,Jackson St. Service Connections,LSL replacement,"$512,000",A*,530,LSLR,,,,"$512,000"
12,15,Frederica,DE0000587,870,Market Street,"Water Main Renewal $328,000",,A*,420,Base,,,Base DAC,"$328,000"
13,28,Frederica,DE0000587,870,LSL inventory,LSL inventory,"$102,000",A*,310,LSLR,,,,"$102,000"
15,31,Lewes BPW,DE0000602,3000,Jones Farm,Tank,"$5,320,000",,290,Corpus,2.00%,20 years,,$0
17,31,Lewes BPW,DE0000602,3000,Savannah Road Bridge,Canal Crossing Water Main,"$2,260,000",,290,Corpus,2.00%,20 years,,$0
19,20,Magnolia,DE0000610,425,Water Production Well,Well Improvements,"$752,100",A*,375,Corpus,2.00%,20 years,,$0
